In [1]:
import numpy as np
from scipy.special import binom
from scipy.optimize import  minimize_scalar

In [2]:
# calculates the grid points for numerical integration on (0,1) as the roots of the shifted Legendre Polynomials
def initializePoints(q):
    arr = np.zeros(q)
    arr[1:q-1] = np.polynomial.polynomial.polyroots(np.polynomial.polynomial.polyder(shiftedLegendre(q-1).coef))
    arr[q-1] = 1
    return arr

# initializes the weights for Gauß-Lobatto integration on (0,1)
def initializeWeights(q, ArrayPoints):
    return [1/(q*(q-1)*np.polynomial.polynomial.polyval(j,shiftedLegendre(q-1).coef)**2) for j in ArrayPoints]

# calculates the n.th shifted Legendre polynomial (on (0,1))
def shiftedLegendre(n):
    arr = np.zeros(n+1)
    for idx in range(n+1):
        arr[idx] = binom(n,idx)*binom(n+idx,idx)*(-1)**(n+idx)
    return np.polynomial.polynomial.Polynomial(arr)
    
# calculate the n-th. basis function for the polynomial approximation space
def functionL(n):
    if(n==0):
        return shiftedLegendre(0)
    elif(n%2 == 1):
        return shiftedLegendre(n+1)
    else:
        return shiftedLegendre(n+1) - shiftedLegendre(1)

# returns an array containing all basis functions    
def createBasis(p):
    return [functionL(j) for j in range(1,p)]

# evaluates the function a at the array ArrayPoints (ArrayPoints[i] in [0,1])
def discretizeFunction(a, ArrayPoints):
    assert (all([(x <= 1 and x >= 0) for x in ArrayPoints]))
    return [a(j) for j in ArrayPoints]

# returns minimum and maximum of a on [0,1]
def MinMax(a): 
    amin = minimize_scalar(a, method = 'bounded',bounds = (0,1)).__getitem__('fun')
    amax = (-1)*minimize_scalar(lambda x: -a(x), method = 'bounded', bounds = (0,1)).__getitem__('fun')
    return [amin,amax]